This Python script is designed to create two tables in a MySQL database for a basic **Bank Management System**. The script uses the `mysql.connector` library to connect to the MySQL database and execute SQL commands.

### Key Components:
1. **Database Configuration**:
   - The `DB_CONFIG` dictionary holds the connection details (host, user, password, database) required to connect to the MySQL database.

2. **Table Creation**:
   - **`accounts` Table**:
     - Stores customer account details, including `account_number` (primary key), `name`, `address`, `phone`, `balance`, `password`, `location_code`, and `bank_code`.
   - **`transactions` Table**:
     - Records all transactions, including `transaction_id` (auto-incremented primary key), `account_number` (foreign key referencing `accounts`), `timestamp`, `transaction_type` (deposit, withdrawal, or transfer), `amount`, and `related_account` (for transfers).

3. **Error Handling**:
   - The script includes a `try-except-finally` block to handle potential errors during database connection or table creation. If an error occurs, it prints the error message.

4. **Connection Management**:
   - The script ensures the database connection is properly closed after executing the queries, even if an error occurs.

### Purpose:
This script sets up the necessary database structure for a banking system, enabling the storage of account information and transaction records. It ensures the tables are created only if they do not already exist, making it safe to run multiple times.

### Usage:
Run this script to initialize the database schema for a banking application. Ensure the MySQL server is running and the provided credentials in `DB_CONFIG` are correct.

In [ ]:
# import random
import sys
import datetime
import pandas as pd
import numpy as np
import pymysql
from pymysql import Error
from decimal import Decimal


# Database Configuration
def get_db_connection():
    try:
        conn = pymysql.connect(
            host='localhost',
            user='root',
            password='Riteshsingh@123',
            database='Bankmanagement_system',
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        return conn
    except Error as e:
        print(f"Error connecting to the database: {e}")
        return None

# Function to create tables
def create_tables():
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS accounts (
                    account_number VARCHAR(12) PRIMARY KEY,
                    name VARCHAR(255) NOT NULL,
                    address VARCHAR(255),
                    phone VARCHAR(20),
                    balance DECIMAL(15, 2) NOT NULL,
                    password VARCHAR(255) NOT NULL,
                    location_code VARCHAR(3) NOT NULL,
                    bank_code VARCHAR(3) NOT NULL
                )
            ''')

            cursor.execute('''
                CREATE TABLE IF NOT EXISTS transactions (
                    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
                    account_number VARCHAR(12) NOT NULL,
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    transaction_type ENUM('DEPOSIT', 'WITHDRAWAL', 'TRANSFER') NOT NULL,
                    amount DECIMAL(15, 2) NOT NULL,
                    related_account VARCHAR(12),
                    FOREIGN KEY (account_number) REFERENCES accounts(account_number)
                )
            ''')

            conn.commit()
            print("Tables created successfully!")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if conn:  # Check if the connection object exists
            cursor.close()
            conn.close()

# Function to generate a unique 12-digit account number
def generate_account_number(location_code, bank_code):
    if not (location_code.isdigit() and len(location_code) == 3):
        print("Error: Location code must be a 3-digit number.")
        return None
    if not (bank_code.isdigit() and len(bank_code) == 3):
        print("Error: Bank code must be a 3-digit number.")
        return None
    unique_number = np.random.randint(100000, 999999)  # Generate 6-digit random number
    return f"{location_code}{bank_code}{unique_number}"

# Function to create an account
def create_account(name, address, phone, location_code, bank_code, balance=Decimal('0')):
    account_number = generate_account_number(location_code, bank_code)
    if not account_number:
        return None
    password = input("Set a Password: ").strip()
    
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO accounts (account_number, name, address, phone, balance, password, location_code, bank_code)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ''', (account_number, name, address, phone, balance, password, location_code, bank_code))
            conn.commit()
            print(f"Account created successfully! Account Number: {account_number}")
            return account_number
    except Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()

# Function to sign in
def sign_in(acc_num):
    password = input("Enter Password: ").strip()
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT password FROM accounts WHERE account_number = %s
            ''', (acc_num,))
            account = cursor.fetchone()
            if account and account['password'] == password:  # Use column name as key
                print("Sign-in successful!")
                return acc_num
            else:
                print("Error: Invalid credentials.")
                return None
    except Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()

# Function to register a user
def register():
    name = input("Enter Name: ").strip()
    address = input("Enter Address: ").strip()
    phone = input("Enter Phone: ").strip()
    location_code = input("Enter Location Code (3 digits): ").strip()
    bank_code = input("Enter Bank Code (3 digits): ").strip()
    try:
        balance = Decimal(input("Enter Initial Balance (default 0): ").strip() or 0)
    except ValueError:
        print("Error: Invalid balance amount. Setting balance to 0.")
        balance = Decimal(0)
    
    acc_num = create_account(name, address, phone, location_code, bank_code, balance)
    if acc_num:
        print("Redirecting to Sign In...")
        sign_in(acc_num)


# Function to deposit money
def deposit(acc_num, amount):
    if amount <= 0:
        print("Error: Deposit amount must be positive.")
        return
    conn = None  # Initialize conn here
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            # Update balance
            cursor.execute('''
                UPDATE accounts 
                SET balance = balance + %s 
                WHERE account_number = %s
            ''', (float(amount), acc_num))
            # Log transaction
            cursor.execute('''
                INSERT INTO transactions (account_number, timestamp, transaction_type, amount)
                VALUES (%s, %s, %s, %s)
            ''', (acc_num, datetime.datetime.now(), 'DEPOSIT', float(amount)))
            conn.commit()
            print("Deposit successful!")
    except Error as e:
        if conn:
            conn.rollback()
        print(f"Error during deposit: {e}")
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()

# Function to withdraw money
def withdraw(acc_num, amount):
    if amount <= 0:
        print("Error: Withdrawal amount must be positive.")
        return
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            # Check balance
            cursor.execute('''
                SELECT balance FROM accounts WHERE account_number = %s
            ''', (acc_num,))
            balance = cursor.fetchone()
            if balance['balance'] < amount:  # Use column name
                print("Error: Insufficient balance.")
                return
            # Update balance
            cursor.execute('''
                UPDATE accounts 
                SET balance = balance - %s 
                WHERE account_number = %s
            ''', (float(amount), acc_num))
            # Log transaction
            cursor.execute('''
                INSERT INTO transactions (account_number, timestamp, transaction_type, amount)
                VALUES (%s, %s, %s, %s)
            ''', (acc_num, datetime.datetime.now(), 'WITHDRAWAL', float(amount)))
            conn.commit()
            print("Withdrawal successful!")
    except Error as e:
        if conn:
            conn.rollback()
        print(f"Error during withdrawal: {e}")
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()
            
# Function to transfer money
def transfer(from_acc, to_acc, amount):
    if amount <= 0:
        print("Error: Transfer amount must be positive.")
        return
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            # Check sender balance
            cursor.execute('''
                SELECT balance FROM accounts WHERE account_number = %s
            ''', (from_acc,))
            balance = cursor.fetchone()['balance']  # Use column name
            if balance < amount:
                print("Error: Insufficient balance.")
                return
            # Update sender balance
            cursor.execute('''
                UPDATE accounts 
                SET balance = balance - %s 
                WHERE account_number = %s
            ''', (float(amount), from_acc))
            # Update receiver balance
            cursor.execute('''
                UPDATE accounts 
                SET balance = balance + %s 
                WHERE account_number = %s
            ''', (float(amount), to_acc))
            # Log transactions
            cursor.execute('''
                INSERT INTO transactions (account_number, timestamp, transaction_type, amount, related_account)
                VALUES (%s, %s, %s, %s, %s)
            ''', (from_acc, datetime.datetime.now(), 'TRANSFER', float(amount), to_acc))
            cursor.execute('''
                INSERT INTO transactions (account_number, timestamp, transaction_type, amount, related_account)
                VALUES (%s, %s, %s, %s, %s)
            ''', (to_acc, datetime.datetime.now(), 'TRANSFER', float(amount), from_acc))
            conn.commit()
            print("Transfer successful!")
    except Error as e:
        if conn:
            conn.rollback()
        print(f"Error during transfer: {e}")
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()

# Function to check account details
def check_account_details(acc_num):
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT * FROM accounts WHERE account_number = %s
            ''', (acc_num,))
            details = cursor.fetchone()
            if details:
                print("\n--- Account Details ---")
                print(f"Account Number: {details['account_number']}")  # Column name
                print(f"Name: {details['name']}")                      # Column name
                print(f"Address: {details['address']}")                # Column name
                print(f"Phone: {details['phone']}")                    # Column name
                print(f"Balance: {details['balance']}")                # Column name
            else:
                print("Error: Account not found.")
    except Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()
# Function to check account balance
def check_balance(acc_num):
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT balance FROM accounts WHERE account_number = %s
            ''', (acc_num,))
            balance = cursor.fetchone()
            if balance:
                print(f"Current Balance: {balance['balance']}")  # Column name
            else:
                print("Error: Account not found.")
    except Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:  # Simple check
            cursor.close()
            conn.close()
            
# Function to view transaction history
def view_transaction_history(acc_num):
    conn = None
    try:
        conn = get_db_connection()
        if conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT timestamp, transaction_type, amount, related_account 
                FROM transactions 
                WHERE account_number = %s
                ORDER BY timestamp DESC
            ''', (acc_num,))
            transactions = cursor.fetchall()
            if not transactions:
                print("No transactions found.")
                return
            print("\n--- Transaction History ---")
            for txn in transactions:
                # Use column names
                print(f"{txn['timestamp']} | {txn['transaction_type']} | Amount: {txn['amount']} | Related: {txn['related_account']}")
    except Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:  # Remove `is_connected` check
            cursor.close()
            conn.close()
# Function to sign out
def sign_out():
    print("Successfully signed out.")

# Main function to handle bank operations
def bank_operations():
    create_tables()  # Ensure tables are created
    while True:
        print("\nBank Account Management System")
        print("1. Register")
        print("2. Sign In")
        print("3. Deposit")
        print("4. Withdraw")
        print("5. Transfer")
        print("6. Check Account Details")
        print("7. Check Account Balance")
        print("8. View Transaction History")
        print("9. Sign Out")
        print("10. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            register()
        elif choice == "2":
            acc_num = input("Enter Account Number: ").strip()
            sign_in(acc_num)
        elif choice in ["3", "4", "5", "6", "7", "8"]:
            acc_num = input("Enter Account Number: ").strip()
            if sign_in(acc_num):
                try:
                    if choice == "3":
                        amount = Decimal(input("Enter Deposit Amount: ").strip())
                        deposit(acc_num, amount)
                    elif choice == "4":
                        amount = Decimal(input("Enter Withdrawal Amount: ").strip())
                        withdraw(acc_num, amount)
                    elif choice == "5":
                        to_acc_num = input("Enter Receiver Account Number: ").strip()
                        amount = Decimal(input("Enter Transfer Amount: ").strip())
                        transfer(acc_num, to_acc_num, amount)
                    elif choice == "6":
                        check_account_details(acc_num)
                    elif choice == "7":
                        check_balance(acc_num)
                    elif choice == "8":
                        view_transaction_history(acc_num)
                except ValueError:
                    print("Error: Invalid input. Please enter a valid amount.")
        elif choice == "9":
            sign_out()
        elif choice == "10":
            print("Exiting the system. Thank you!")
            break
        else:
            print("Invalid choice. Please try again.")

# Start the application
if __name__ == "__main__":
    bank_operations()

Tables created successfully!

Bank Account Management System
1. Register
2. Sign In
3. Deposit
4. Withdraw
5. Transfer
6. Check Account Details
7. Check Account Balance
8. View Transaction History
9. Sign Out
10. Exit


Enter your choice:  3
Enter Account Number:  788877778702
Enter Password:  9000


Sign-in successful!


Enter Deposit Amount:  100


Deposit successful!

Bank Account Management System
1. Register
2. Sign In
3. Deposit
4. Withdraw
5. Transfer
6. Check Account Details
7. Check Account Balance
8. View Transaction History
9. Sign Out
10. Exit


...

Transfer successful!

Bank Account Management System
1. Register
2. Sign In
3. Deposit
4. Withdraw
5. Transfer
6. Check Account Details
7. Check Account Balance
8. View Transaction History
9. Sign Out
10. Exit


Enter your choice:  6
Enter Account Number:  788877778702
Enter Password:  9000


Sign-in successful!

--- Account Details ---
Account Number: 788877778702
Name: snehal
Address: kolkata
Phone: 8773678893
Balance: 890100.00

Bank Account Management System
1. Register
2. Sign In
3. Deposit
4. Withdraw
5. Transfer
6. Check Account Details
7. Check Account Balance
8. View Transaction History
9. Sign Out
10. Exit


Enter your choice:  7
Enter Account Number:  788877778702
Enter Password:  9000


Sign-in successful!
Current Balance: 890100.00

Bank Account Management System
1. Register
2. Sign In
3. Deposit
4. Withdraw
5. Transfer
6. Check Account Details
7. Check Account Balance
8. View Transaction History
9. Sign Out
10. Exit
